## Notebook discovery gneral Student Enrollment.
Review codes of example student enrollmen and understanding itt

Sourcesold url: :

https://gurobi-machinelearning.readthedocs.io/en/stable/mlm-examples/student_admission.htnew url: ml


https://gurobi-machinelearning.readthedocs.io/en/stable/auto_examples/example2_student_admission.html#sphx-glr-auto-examples-example2-student-admission-py

### 0. Packages

In [ ]:
# ## install gurobi packages

# !pip install gurobipy
# !pip install --upgrade gurobipy
# !pip install gurobi-machinelearning
# !pip install gurobipy-pandas

In [ ]:
!pip show gurobipy

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from scipy.stats import pointbiserialr, spearmanr

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

import sys

# package gurobi
import gurobipy as gp
from gurobi_ml import add_predictor_constr
import gurobipy_pandas as gppd

### 2. Setear licencia

In [ ]:
# import os
# path_licencia_gurobi = "gurobi.lic"
# os.environ ["GRB_LICENSE_FILE"] = path_licencia_gurobi
# print(os.environ["GRB_LICENSE_FILE"])

In [ ]:
# crear modelo con licencia
modelo_prueba = gp.Model('Modelo Prueba')
modelo_prueba

### 3. Explicación data y variables

#### CONTEXTO PROBLEMA

Tengo un conjunto de estudiantes que desean ingresar a la universidad, tiene un carrera en el colegio y además dan pruebas para entrar a la universidad (esto se ve reflejado un variables SAT y GPA), además la universidad (de acuerdo a criterios que halla tenido esta decide ofrecerle a cada estudiante una cierta cantidad de dinero de beca, definido como variable MERIT). Luego, el estudiante en base a criterios que el tenga decide ingresar o no la universidad (visto como variable target ENROLL).

El objetivo de la universidad es maximizar la cantidad de alumnos que ingresan sujeto a un cierto presupuesto que ellos tienen.

**EXPLICACIÓN:**
- Que el estudiante ingresa a la universidad o no (y) está dado por una función que mapea el monto de la beca (X), GPA y SAT
- Cada estudiante puede recibir como máximo 2.5 como beca (merit). Un máximo de 2.5k de beca
- Existe un presupuesto total para becas que está dado por la "cantidad de alumnos que postulan" multiplicado por un cierto factor de la forma 0.2n
- Quiero maximizar la cantidad de alumnos que ingresan a la universidad decidiendo qué monto de beca asignarle

#### LIST OF ALL VARIABLES IN THE DATA

- **SAT**: The SAT is an entrance exam used by most colleges and universities to make admissions decisions. The SAT is a multiple-choice, pencil-and-paper test created and administered by the College Board.
- **GPA**: Grade Point Average
- **merit**: Amounth of money offered as a scholarship for a student (I think probably according the SAT and GPA). The column is named merit because probably I think according the SAT and GPA of a student the University offers more or minus money
- **enroll**: binary variable. 1 the student ingress to the university. 0 The student doens't ingress to the university

#### CLASIFIACIÓN VARIABLES
Desde el punto de vista de la universidad, las variables se pueden clasificar en:

**Variable no controlables:**
- SAT
- GPA

**Variables controlables:**
- merit (cantidad de dinero que ofrecen)

**Variable resultante:**
- enroll

### 4. Read data

In [ ]:
# Read Data: Base URL for retrieving data
janos_data_url = "https://raw.githubusercontent.com/INFORMSJoC/2020.1023/master/data/"
historical_data = pd.read_csv(
    janos_data_url + "college_student_enroll-s1-1.csv", index_col=0
)

In [ ]:
historical_data.head()

In [ ]:
# classify our features between the ones that are fixed and the ones that will be
# part of the optimization problem
features = ["merit", "SAT", "GPA"]
target = "enroll"
features_target = features + [target]

### 5. EDA

#### 5.1 General eda

In [ ]:
# descriptive statistic
historical_data.describe()

In [ ]:
# histograms
for feature in features_target:
    historical_data[feature].hist()
    plt.title(feature)
    plt.show()

#### 5.2 correlations

In [ ]:
correlation_matrix = historical_data.corr()

# plt.figure(figsize=(10, 8))
# sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
# plt.title('Correlation Matrix')
# plt.show()

correlation_matrix

Insights:
- gpa y sat están altamente correlacionados, demasiado (0.96)
- merit tiene cierta relación con sat y gpa. A un sat alto también habrá un merit (beca) mayor. Sin embargo, se esperaría una correlación mayor pero no es el caso. En los gráficos a continuación al separar merit y no merit también se puede observar el porqué de la baja correlación (bajo lo que se esperaría)
- enroll is binary variable. Doesn't apply this correlation

#### 5.3 correlations features(continuos) vs target (categorical)

In [ ]:
# Calcular la correlación de punto biserial entre la variable binaria y las variables continuas
correlations_pb = {}
for column in historical_data.columns:
    if column != target:
        correlation, p_value = pointbiserialr(historical_data[target], historical_data[column])
        correlations_pb[column] = correlation

# Opcional: Puedes utilizar el coeficiente de correlación de rango de orden (Spearman) para variables no lineales
# correlations_spearman = historical_data.corr(method='spearman')['target']

# Crear un DataFrame con las correlaciones
correlations_df = pd.DataFrame(list(correlations_pb.items()), columns=['Variable', 'Point Biserial Correlation'])
correlations_df.set_index('Variable', inplace=True)

# Mostrar el DataFrame con las correlaciones
print(correlations_df)

# Crear un mapa de calor (heatmap) de las correlaciones
# plt.figure(figsize=(10, 6))
# sns.heatmap(correlations_df.transpose(), annot=True, cmap='coolwarm', linewidths=0.5)
# plt.title('Correlaciones con la Variable Target')
# plt.show()

#### 5.3 Zoom merit. because some values in the histogram are zero?

In [ ]:
# divide data merit and no merit
data_no_merit = historical_data[historical_data['merit']==0]
data_merit = historical_data[historical_data['merit']!=0]

# number merit and no merit
number_of_students_no_merit = historical_data[historical_data['merit']==0].shape[0]
number_of_students_merit = historical_data[historical_data['merit']!=0].shape[0]

In [ ]:
# histograms
def plot_hist_merit_no_merit(df_no_merit, df_merit, variable):
    """
    Plot histogram of a feature divide into dataframe with merit and no merit
    """
    plt.hist(df_no_merit[variable], label = 'NO merit', alpha = 0.3, color = 'gray')
    plt.hist(df_merit[variable], label = 'merit', alpha = 0.3, color = 'orange')
    plt.legend()
    plt.title(feature)
    plt.show()

for feature in features_target:
    plot_hist_merit_no_merit(df_no_merit = data_no_merit, 
                             df_merit = data_merit, 
                            variable = feature)

Insights:
- con no merit se observa (merit = 0) se observa que la cola del histograma de merit efectivamente era porque el valor era cero
- es más frecuente un merit = 0 con un SAT bajo (de acuerdo a la intuición)
- es más frecuente un merit = 0 con un GPA bajo (de acuerdo a la intuición)
- se pueden hacer más análisis pero no es el objetivo de este notebook
- Si hay merit este parte desde 0.5k hasta los 2.5 valor tope de acuerdo a la explicación del problema

**Conclusiones:**
- El monto de la beca que como variable de decisión puede tomar valores entre 0 y 2.5, pero en los datos, los valores se mueven desde 0.5 hasta 2.5. Lo que puede generar un riesg en caso de que se quisiera probar el efecto de asignarle a alguien por ejemplo 0.3 de becao

### 6. Model
Predecir si el estudiante va a ingresar a no a la universidad de acuerdo a SAT, GPA y merit

In [ ]:
# split data
X_train, X_test, y_train, y_test = train_test_split(
     historical_data.loc[:, features],
     historical_data.loc[:, target],
     test_size=0.2,
     random_state=42)

In [ ]:
X_train.head()

In [ ]:
# Run our regression
scaler = StandardScaler()
regression = LogisticRegression(random_state = 42)
pipe = make_pipeline(scaler, regression)
pipe.fit(X = X_train, y = y_train)

### 7. Evaluación modelo

In [ ]:
y_pred_test = pipe.predict(X_test)

In [ ]:
# accuracy
accuracy_score(y_true = y_test,
               y_pred = y_pred_test
               )

In [ ]:
# confusion matrix datos test

confusion_matrix(y_true = y_test,
                 y_pred = y_pred_test
                 )

### 8. Optimization model
Luego de tener un modelo que dado: SAT, GPA y merit pueda calcular si el alumno va a ingresar a la universidad o no (utilizando datos históricos, ej postulaciones de la universidad de lo últimos 5 años), ahora con datos nuevos donde está la información de las variables no controlables SAT y GPA, ej de las postulaciones a la universidad de este año, la universidad busca conocer qué MERIT (beca económica) ofrecerle a este conjunto de estudiantes para maximar la cantidad de alumnos que entran con el presupuesto de becas que cuentanmente

CARGAR DATA
**Esta data no contiene las variables de decisión del modelo.** En este caso, contiene solo SAT y GPA ya que merit y enroll son variables de decisión X e y respectivamente

PARÁMETROS
- Los datos obtenidos del dataset SAT, GPA
- Presupuesto

VARIABLES DE DECISIÓN
- **X**: en los datos, es la columna: **merit**. Variable continua. Es la cantidad de dinero que se le asigna como beca. Dinero medido en K.
- **y**: en los datos, es la columna: **enroll**. variable binaria, si el estudiante ingresa a la universidad

#### 8.1 Cargar data

In [ ]:
# Retrieve new data used to build the optimization problem
studentsdata = pd.read_csv(janos_data_url + "college_applications6000.csv", index_col=0)
studentsdata

In [ ]:
# visualizar data para optimizador vs data historica - SAT
plt.hist(studentsdata['SAT'], alpha = 0.2, label = 'data_para_optimizador')
plt.hist(historical_data['SAT'], alpha = 0.2, label = 'data_historical')
plt.legend()
plt.title('SAT')
plt.show()

In [ ]:
# visualizar data para optimizador vs data historica - GPA
plt.hist(studentsdata['GPA'], alpha = 0.2, label = 'data_para_optimizador')
plt.hist(historical_data['GPA'], alpha = 0.2, label = 'data_historical')
plt.legend()
plt.title('GPA')
plt.show()

In [ ]:
# TODO:
# validar igualdad de distribuciones
# https://towardsdatascience.com/comparing-sample-distributions-with-the-kolmogorov-smirnov-ks-test-a2292ad6fee5

### 8.2 Samplear por licencia gratuita
Samplear datos por licencia gratuita que admite solo 250 índices

In [ ]:
nstudents = 25

# Select randomly nstudents in the data
studentsdata = studentsdata.sample(nstudents)

#### 8.3 Create Optimization model
Since our data is in pandas data frames, we use the **package gurobipy-pandas to help create the variables directly using the index of the data frame**

In [ ]:
# Start with classical part of the model
m = gp.Model("Student Enrollment Model")
m

#### 8.4 Agregar variables de decisión

In [ ]:
# Agregar variable de decisión: y
# The y variables are modeling the probability of enrollment of each student. They are indexed by students data
y = gppd.add_vars(m, studentsdata, name='enroll_probability')
y

In [ ]:
# TODO YO: COMO RECONOCE QUE ES UNA VARIABLE BINARIA?

In [ ]:
# Agregar variable de decisión: X - agregarla directamente en el dataframe. 
# ESTO PORQUE PARA EL MODELO DE MACHINE LEARNING ES MAS FÁCIL PASAR EL DATAFRAME CON TODAS LAS FEATURES QUE NECESITA PARA HACER LA INFERENCIA
# se le pasa el dataframe con las features que no son variables de decisión y se crean las columnas con las features que sí son variables de decisión


# We add to studentsdata a column of variables to model the "merit" feature. Those variable are between 0 and 2.5.
# They are added directly to the data frame using the gppd extension.
studentsdata = studentsdata.gppd.add_vars(m, lb=0.0, ub=2.5, name='merit')

In [ ]:
studentsdata.head()

In [ ]:
# Definir variable de decisión X. AQUI SIMPLEMENTE SE DEFINE UNA VARIABLE DE PYTHON "x" sin las otras variables del dataframe porque para definir una restricción se hace de la forma x.sum(),
# SIN EMBARGO ESTO ES TOTALMENTE INNECESARIO Y SE PUEDE OMITIR ESTA DEFICIÓN DE VARIABLES PARA EVITAR CONFUNDIR

# We denote by x the (variable) "merit" feature
x = studentsdata.loc[:, "merit"]
x

In [ ]:
# ordenar dataframe en el mismo orden de feautures utilizado para entrenar el modelo
# Make sure that studentsdata contains only the features column and in the right order
studentsdata = studentsdata.loc[:, features]
studentsdata

In [ ]:
# "compilar" el modelo de optimización - cargando las variables de decisión - 6000 estudiantes - 6000 elementos en el conjunto i - Variables de decisión Xi = 6000 y yi = 6000 -> 12000 variables de decisión
m.update()
m

In [ ]:
# Let's look at our features dataframe for the optimization
studentsdata[:10]

#### 8.4 Agregar función objetivo y restricciones

In [ ]:
# AGREGAR FUNCIÓN OBJETIVO. Al tomar como origen un dataframe, permite escribir sumatorias de variables como si fueran un dataframe
y.sum()

In [ ]:
# funcion objetivo del modelo
m.setObjective(y.sum(), gp.GRB.MAXIMIZE)

In [ ]:
### agregar restricciones del modelo - presupuesto para becas (ojo solo es una restricción)

len_students = studentsdata.shape[0] # calcular la cantidad de estudiantes
m.addConstr(x.sum() <= 0.2 * len_students)

In [ ]:
0.2 * len_students # ver lado derecho de restricción, presupuesto disponible

In [ ]:
# ver modelo antes de actualizarse - no tiene cargada las restricciones ni la función objetivo
m

In [ ]:
# actualizar modelo
m.update()

In [ ]:
# ver modelo actualizado - se agregó SOLO UNA RESTRICCIÓN: limite de presupuesto
m

#### 8.6 Agregar restricción dada por el modelo de ML
predicción de prob de unirse estudiantes dado GPA, SAT y merit

**"add_predictor_const"**

Documentación códigos: https://github.com/Gurobi/gurobi-machinelearning/blob/main/src/gurobi_ml/add_predictor.py

Se deben de definir los siguientes parámetros de entrada:

    gp_model : :gurobipy:`model`
            The gurobipy model where the predictor should be inserted.
    predictor:
        The predictor to insert.
    input_vars : mvar_array_like
        Decision variables used as input for predictor in gp_model.
    output_vars : mvar_array_like, optional
        Decision variables used as output for predictor in gp_model.

In [ ]:
# definir variable de entrada
pred_constr = add_predictor_constr(
    m, # model gurobi
    pipe, # predictor - artefacto modelo ml
    studentsdata, # input_var - dataframe que contiene las instancias de entrada del modelo de optimización. valores numéricos y variables de decisión
    y, # output_var - variables de decisión
    output_type="probability_1"
)

In [ ]:
studentsdata

In [ ]:
pred_constr

In [ ]:
# no se necesita hacer un model.update(), por lo que se ve, se actualiza de forma inmediata agregando las restricciones
m

In [ ]:
# estadísticas de los modelos agregados como restricciones
# referencia: son 6000 estudiantes, por lo tanto son 6000 elementos en el conjunto i
pred_constr.print_stats()

### 8.7 Optimizar

In [ ]:
m.optimize()

### 8.8 Consideraciones y mejoras para regresiones logaritmicas
Remember that for the logistic regression, Gurobi does a piecewise-linear approximation of the logistic function. We can therefore get some significant errors when comparing the results of the Gurobi model with what is predicted by the regression.

In [ ]:
print(
    "Maximum error in approximating the regression {:.6}".format(
        np.max(pred_constr.get_error())
    )
)

The error we get might be considered too large, but we can use Gurobi parameters to tune the piecewise-linear approximation made by Gurobi (at the expense of a harder models).

The specific parameters are explained in the documentation of Functions Constraints in Gurobi’s manual.

We can pass those parameters to the add_predictor_constr function in the form of a dictionary with the keyword parameter pwd_attributes.

Now we want a more precise solution, so we remove the current constraint, add a new one that does a tighter approximation and resolve the model.

https://www.gurobi.com/documentation/9.1/refman/constraints.html#subsubsection:GenConstrFunction

In [ ]:
pred_constr.remove()

pwl_attributes = {
    "FuncPieces": -1,
    "FuncPieceLength": 0.01,
    "FuncPieceError": 1e-5,
    "FuncPieceRatio": -1.0,
}
pred_constr = add_predictor_constr(
    m, pipe, studentsdata, y, output_type="probability_1", pwl_attributes=pwl_attributes
)

m.optimize()

In [ ]:
print(
    "Maximum error in approximating the regression {:.6}".format(
        np.max(pred_constr.get_error())
    )
)

### 8.9 Resultados

In [ ]:
pred_constr.input_values

In [ ]:
pred_constr.output_values

In [ ]:
pd.DataFrame(pred_constr.output_values)

In [ ]:
pd.DataFrame(pred_constr.output_values).hist()

In [ ]:
x

In [ ]:
y

In [ ]:
x.iloc[0]

In [ ]:
y.iloc[0]

In [ ]:
# obtener un valor individual de las variables de decisión
y.iloc[0].X

In [ ]:
# obtener un valor individual de las variables de decisión
x.iloc[0].X

In [ ]:
# obtener valores desde un for
x_values = []
for index in range(x.shape[0]):
  x_values.append(x.iloc[index].X)

x_values

In [ ]:
print(f"Optimal objective value: {m.objVal}")

In [ ]:
# # para obtener los valores de la variable de decisión como serie de pandas bien ----> AL FINAL ESTA ES LA MEJOR FORMA PARA OBTENER LOS VALORES
# DE LAS VARIABLES DE DECISIÓN DEL MODELO DE OPTIMIZACIÓN
x.gppd.X

# IMPORTANTE REVISAR ERROR

In [ ]:
### TODO YO:
# LAS VARIABLES DE DECISIÓN "Y" TOMAN VALORES NUMÉRICOS EN LUGAR DE NÚMEROS BINARIOS
# REVISAR POR QUÉ SIENDO QUE EL MODELO PREDICE VALORES BINARIOS

In [ ]:
y_pred_test

# NOTE: EL PROBLEMA CON ESTE EJEMPLO ES QUE LA OPTIMIZACIÓN ES PARA UN CONJUNTO DE DATOS, POR EJEMPLO, DADO 100 ESTUDIANTES QUE POSTULAN ESTE AÑO, QUÉ CANTIDAD DE BECAS OFRCERLE A CADA ESTUDIANTE DADO QUE TENGO UN PRESUPUESTO QUE DEPENDE DE LA CANTIDAD DE ESTUDIANTES

# ---> NO ES POSIBLE PREDECIR QUÉ BECA OFRCERLE A UN ESTUDIANTE INDIVIDUAL QUE POSTULA <-------